- n of unique songs
- first song and first discovered artist
- minutes
- n1 artist w/amount of hours
- top artists, top songs, top genres
- top 100
- subgenre
- oldest song

In [3]:
import pandas as pd
import numpy as np
import pylast
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import time
import datetime
import re
from IPython.display import clear_output


In [11]:
from config import LASTFM_API_KEY, LASTFM_SHARED_SECRET, LASTFM_PASSWORD

In [12]:
# In order to perform a write operation you need to authenticate yourself
username = "alexing10"
password_hash = pylast.md5(LASTFM_PASSWORD)

network = pylast.LastFMNetwork(api_key=LASTFM_API_KEY, api_secret=LASTFM_SHARED_SECRET,
                               username=username, password_hash=password_hash)
user = network.get_authenticated_user()

In [4]:
d = datetime.date(2018,1,1)

unixtime = time.mktime(d.timetuple())
tracks = user.get_recent_tracks(limit=None, time_from=unixtime)

In [5]:
first_elem = tracks[0]
print(type(first_elem), len(tracks))

<class 'pylast.PlayedTrack'> 14812


In [6]:
def print_track(a_track):
    print(".artist: ", a_track.track.artist)
    album = str(a_track.track.get_album())
    if not album:
        album = ''
    print(".get_album(): ", re.sub(r"^[^-]*- ", "", album))
    print(".get_duration(): ",a_track.track.get_duration())
    print(".get_listener_count(): ",a_track.track.get_listener_count())
    print(".get_mbid(): ",a_track.track.get_mbid())
    print(".get_playcount(): ",a_track.track.get_playcount())
    print(".get_url(): ",a_track.track.get_url())
    print(".info: ",a_track.track.info)
    print(".title: ",a_track.track.title)

print_track(first_elem)

.artist:  Marilina Bertoldi
.get_album():  Prender un Fuego
.get_duration():  188000
.get_listener_count():  227
.get_mbid():  None
.get_playcount():  1351
.get_url():  https://www.last.fm/music/marilina%2bbertoldi/_/nunca
.info:  {}
.title:  Nunca


In [7]:
print_track(tracks[568])

.artist:  Riff
.get_album():  None
.get_duration():  0
.get_listener_count():  31
.get_mbid():  None
.get_playcount():  46
.get_url():  https://www.last.fm/music/riff/_/sube%2ba%2bmi%2bvoiture%2b-%2ben%2bvivo
.info:  {}
.title:  Sube a Mi Voiture - En Vivo


In [17]:
def get_data(a_track):
    data = []
    data.append(str(a_track.track.artist))
    data.append(a_track.track.title)
    try:
        album = str(a_track.track.get_album())
        if not album:
            album = ''
    except:
        album = ''
    data.append(re.sub(r"^[^-]*- ", "", album))
    try:
        duration = a_track.track.get_duration()
    except:
        duration = ''
    data.append(duration)
    try:
        listener_count = a_track.track.get_listener_count()
    except:
        listener_count = ''
    data.append(listener_count)
    try:
        mbid = a_track.track.get_mbid()
    except:
        mbid = ''
    data.append(mbid)
    try:
        playcount = a_track.track.get_playcount()
    except:
        playcount = ''
    data.append(playcount)
    try:
        url = a_track.track.get_url()
    except:
        url = ''
    data.append(url)
    data.append(a_track.track.info)
    data.append(a_track.playback_date)
    data.append(a_track.timestamp)
    return data

In [18]:
data = {}
amount_of_tracks = len(tracks)
for i, a_track in enumerate(tracks):
    data[i] = get_data(a_track)
    print("Got data from %d/%d tracks" % (i+1, amount_of_tracks))
    clear_output(wait=True)

Got data from 14812/14812 tracks


In [19]:
df = pd.DataFrame.from_dict(data, orient='index')
df.columns=['artist', 'title', 'album', 'duration', 'listener_count', 'mbid', 'playcount', 'url', 'info', 'date', 'timestamp']
df.head()

,artist,title,album,duration,listener_count,mbid,playcount,url,info,date,timestamp
0,Marilina Bertoldi,Nunca,Prender un Fuego,188000,228,None,1366,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:43",1544298210
1,Marilina Bertoldi,La Casa de A,None,0,243,None,1688,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:40",1544298014
2,Marilina Bertoldi,Fumar de día,Prender un Fuego,178000,350,None,2149,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:37",1544297834
3,Marilina Bertoldi,O No?,Prender un Fuego,181000,242,None,1343,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:34",1544297653
4,The 1975,Inside Your Mind,ネット上の人間関係についての簡単な調査 [Explicit],230000,20179,None,63956,https://www.last.fm/music/the%2b1975/_/inside%...,{},"08 Dec 2018, 18:59",1544295551


In [20]:
df.to_csv("data/data.csv")

Repair

In [44]:
df['track'] = None
for i, a_track in enumerate(tracks):
    if df.at[i, 'timestamp'] == a_track.timestamp:
        df.at[i, 'track'] = a_track

In [47]:
df['track'].isnull().sum()

0

In [85]:
amount_of_tracks = len(df)
for i, row in df.iterrows():
    try:
        if row['album'] == 'None':
            album = str(row['track'].track.get_album())
            df.at[i, 'album'] = re.sub(r"^[^-]*- ", "", album)
        if row['duration'] == 0:
            df.at[i, 'duration'] = row['track'].track.get_duration()
        if row['listener_count'] == '':
            df.at[i, 'listener_count'] = row['track'].track.get_listener_count()
        if row['mbid'] == None:
            df.at[i, 'mbid'] = row['track'].track.get_mbid()           
        if row['playcount'] == '':
            df.at[i, 'playcount'] = row['track'].track.get_playcount()           
    except:
        continue
    print("Tried to repair data from %d/%d tracks" % (i+1, amount_of_tracks))
    clear_output(wait=True)
print("Amount of 'None' albums left: %d" % ((df.album == 'None').sum()))
print("Amount of 0 duration left: %d" % ((df.duration == 0).sum()))
print("Amount of '' listener_count left: %d" % ((df.listener_count == '').sum()))
print("Amount of '' playcount left: %d" % ((df.playcount == '').sum()))


Amount of 'None' albums left: 2604
Amount of 0 duration left: 2598
Amount of '' listener_count left: 3
Amount of '' playcount left: 3


Didn't work. Maybe query spotify?

Use first result of search

In [14]:
from config import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET

client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,
                                                      client_secret=SPOTIFY_CLIENT_SECRET)
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET, redirect_uri="http://localhost:8888/")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, auth=token)


In [172]:
errors = []
amount_of_tracks = len(df)
for i, row in df.iterrows():
    if row['album'] == 'None' or row['duration'] == 0:
        try:
            results = sp.search(q=u' '.join([row['artist'], row['title']]), type='track', market='AR')
        except:
            continue
        if len(results['tracks']['items']) == 0:
            if row['artist'] == 'Sandman Project' and row['title'] == 'Circies':
                row['title'] = df.at[i, 'title'] = 'Circles'
                
            query = re.sub(r'\s-.*$','',u' '.join([row['artist'], row['title']]))
            results = sp.search(q=query, type='track', market='AR')
            print(results)
            print(' '.join([row['artist'], row['title']]))
        try:
            if row['album'] == 'None':
                df.at[i, 'album'] = results['tracks']['items'][0]['album']['name']
            if row['duration'] == 0:
                df.at[i, 'duration'] = results['tracks']['items'][0]['duration_ms']      
        except:
            errors.append((i, row))
            continue
            
    print("Tried to repair data from %d/%d tracks" % (i+1, amount_of_tracks))
    clear_output(wait=True)
print("Amount of 'None' albums left: %d" % ((df.album == 'None').sum()))
print("Amount of 0 duration left: %d" % ((df.duration == 0).sum()))
print("Problems:")
for an_error in errors:
    print("Detected error in index: %d with %s - %s" % (an_error[0], an_error[1]['artist'], an_error[1]['title']))

Amount of 'None' albums left: 40
Amount of 0 duration left: 33
Problems:
Detected error in index: 7500 with Gilad Hekselman - Suite For Sweets
Detected error in index: 8124 with Dirty Projectors - (I Wanna) Feel It All (feat. Dear Nora)
Detected error in index: 8125 with Dirty Projectors - You're The One (feat. Robin Pecknold & Rostam)
Detected error in index: 8129 with Dirty Projectors - Zombie Conqueror (feat. Empress Of)
Detected error in index: 8130 with Dirty Projectors - I Feel Energy (feat. Amber Mark)
Detected error in index: 8133 with Dirty Projectors - Right Now (feat. Syd)
Detected error in index: 8166 with Dirty Projectors - Right Now (feat. Syd)
Detected error in index: 8756 with Seu Jorge - O Astronauta De Mármore - Starman
Detected error in index: 9207 with John Lennon - Medley: Rip It Up/Ready Teddy - 2010 - Remaster
Detected error in index: 9373 with Paul McCartney - Ain't No Sunshine - Live On MTV Unplugged
Detected error in index: 9377 with Paul McCartney - Hi-Heel S

Manually fixing at last

In [185]:
df.at[7500, 'album'] = 'SplitLife'               # with Gilad Hekselman - Suite For Sweets
df.at[8124, 'album'] = 'Lamp Lit Prose'             # with Dirty Projectors - (I Wanna) Feel It All (feat. Dear Nora)
df.at[8125, 'album'] = 'Lamp Lit Prose'             #  with Dirty Projectors - You're The One (feat. Robin Pecknold & Rostam)
df.at[8129, 'album'] = 'Lamp Lit Prose'             #  with Dirty Projectors - Zombie Conqueror (feat. Empress Of)
df.at[8130, 'album'] = 'Lamp Lit Prose'             #  with Dirty Projectors - I Feel Energy (feat. Amber Mark)
df.at[8133, 'album'] = 'Lamp Lit Prose'             #  with Dirty Projectors - Right Now (feat. Syd)
df.at[8166, 'album'] = 'Lamp Lit Prose'             #  with Dirty Projectors - Right Now (feat. Syd)
df.at[8756, 'album'] = 'The Life Aquatic Studio Sessions'             # with Seu Jorge - O Astronauta De Mármore - Starman
df.at[9207, 'album'] = "Rock 'N' Roll"            #  with John Lennon - Medley: Rip It Up/Ready Teddy - 2010 - Remaster
df.at[9373, 'album'] = 'Unplugged (The Official Bootleg)'       #        with Paul McCartney - Ain't No Sunshine - Live On MTV Unplugged
df.at[9377, 'album'] = 'Unplugged (The Official Bootleg)'       #        with Paul McCartney - Hi-Heel Sneakers - Live On MTV Unplugged
df.at[9378, 'album'] = 'Unplugged (The Official Bootleg)'       #        with Paul McCartney - She's a Woman - Live On MTV Unplugged
df.at[9381, 'album'] = 'Unplugged (The Official Bootleg)'       #       with Paul McCartney - San Francisco Bay Blues - Live On MTV Unplugged
df.at[9383, 'album'] = 'Unplugged (The Official Bootleg)'       #       with Paul McCartney - Blue Moon of Kentucky - Live On MTV Unplugged
df.at[9385, 'album'] = 'Unplugged (The Official Bootleg)'       #        with Paul McCartney - I Lost My Little Girl - Live On MTV Unplugged
df.at[9386, 'album'] = 'Unplugged (The Official Bootleg)'       #        with Paul McCartney - Be-Bop-A-Lula - Live On MTV Unplugged
df.at[9424, 'album'] = 'Wings Over America'      #         with Paul McCartney - Venus And Mars / Rock Show / Jet - 2013 Remaster
df.at[10187, 'album'] = ''       #       with Static & Ben El - Otw
df.at[10396, 'album'] = 'Jardín'        #      with Gabriel Garzón-Montano - Bombo Fabrica
df.at[10583, 'album'] = 'Leyli'         #     with Leyli - Nishbati Lachzor Elecha
df.at[10584, 'album'] = 'Leyli'         #     with Leyli - Holchim Al Ze
df.at[10585, 'album'] = 'Leyli'         #     with Leyli - Hi
df.at[10586, 'album'] = 'Leyli'         #     with Leyli - Ze Kvar Nikar
df.at[10587, 'album'] = 'Leyli'         #     with Leyli - Rak Shetikach Oti Ita
df.at[10588, 'album'] = 'Leyli'         #    with Leyli - Naki Umeluchlach
df.at[10589, 'album'] = 'Leyli'         #     with Leyli - Tzafona
df.at[10590, 'album'] = 'Leyli'         #     with Leyli - Sham Me'ever Lapsanter
df.at[10591, 'album'] = 'Leyli'         #    with Leyli - BeYamim
df.at[11346, 'album'] = 'SplitLife'     #         with Gilad Hekselman - Suite For Sweets
df.at[12072, 'album'] = 'SplitLife'     #         with Gilad Hekselman - Suite For Sweets
df.at[12088, 'album'] = 'SplitLife'     #         with Gilad Hekselman - When Will The Blues Leave
df.at[12089, 'album'] = 'SplitLife'     #        with Gilad Hekselman - Suite For Sweets
df.at[12092, 'album'] = 'SplitLife'     #        with Gilad Hekselman - Hello Who Is It?

mean = int(np.round(np.mean(df[df['duration'] != '']['duration'])))
for i in [x[0] for x in errors]:
    df.at[i, 'duration'] = mean

In [186]:
print("Amount of 'None' albums left: %d" % ((df.album == 'None').sum()))
print("Amount of 0 duration left: %d" % ((df.duration == 0).sum()))

Amount of 'None' albums left: 12
Amount of 0 duration left: 5


In [202]:
df[df.album == 'None'].index

Int64Index([ 9020, 11197, 11460, 11461, 11838, 11866, 12236, 12237, 12455,
            12540, 12819, 13641],
           dtype='int64')

In [203]:
df.at[9020, 'album'] = '22, A Million'
df.at[11197, 'album'] = 'Band On The Run'
df.at[11460, 'album'] = '' 
df.at[11461, 'album'] =  ''
df.at[11838, 'album'] =  'Perdido Dia Gris'
df.at[11866, 'album'] = "Outlandos D'Amour"
df.at[12236, 'album'] = '' 
df.at[12237, 'album'] = '' 
df.at[12455, 'album'] = ''
df.at[12540, 'album'] = '' 
df.at[12819, 'album'] = ''
df.at[13641, 'album'] = 'Sueño Stereo'
for i in df[df.duration == 0].index:
    df.at[i, 'duration'] = mean

In [314]:
for i in df[df.duration == ''].index:
    df.at[i, 'duration'] = mean

In [204]:
print("Amount of 'None' albums left: %d" % ((df.album == 'None').sum()))
print("Amount of 0 duration left: %d" % ((df.duration == 0).sum()))

Amount of 'None' albums left: 0
Amount of 0 duration left: 0


### FINALLY

In [205]:
df.head()

,artist,title,album,duration,listener_count,mbid,playcount,url,info,date,timestamp,track
0,Marilina Bertoldi,Nunca,Prender un Fuego,188000,228,None,1366,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:43",1544298210,"(Marilina Bertoldi - Nunca, Prender un Fuego, ..."
1,Marilina Bertoldi,La Casa de A,Prender un Fuego,196000,243,None,1688,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:40",1544298014,"(Marilina Bertoldi - La Casa de A, Prender un ..."
2,Marilina Bertoldi,Fumar de día,Prender un Fuego,178000,350,None,2149,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:37",1544297834,"(Marilina Bertoldi - Fumar de día, Prender un ..."
3,Marilina Bertoldi,O No?,Prender un Fuego,181000,242,None,1343,https://www.last.fm/music/marilina%2bbertoldi/...,{},"08 Dec 2018, 19:34",1544297653,"(Marilina Bertoldi - O No?, Prender un Fuego, ..."
4,The 1975,Inside Your Mind,ネット上の人間関係についての簡単な調査 [Explicit],230000,20179,None,63956,https://www.last.fm/music/the%2b1975/_/inside%...,{},"08 Dec 2018, 18:59",1544295551,"(The 1975 - Inside Your Mind, A Brief Inquiry ..."


In [348]:
# df.to_csv("data/data.csv")

In [6]:
# df = pd.read_csv("data/data.csv").drop("Unnamed: 0", axis=1)


## Let's start the analysis

### 1) "Este año, escuchaste 8.486 canciones diferentes en Spotify." <font color='red'>FALSE</font>

In [7]:
unique_songs = df.drop_duplicates(subset=['artist', 'title', 'album']).shape[0]
unique_songs

6693

In [8]:
print("There's a difference of %d in the analysis" % (8486 - unique_songs))

There's a difference of 1793 in the analysis


### 2) "You started 2018 by listening to Life Wasted - Brendan O' Brien Mix by Pearl Jam and the first artist you discovered was Faces." <font color='green'>TRUE</font> / <font color='red'>FALSE</font>

In [9]:
df.loc[df['timestamp'].idxmin()]

artist                                                    Pearl Jam
title                             Life Wasted - Brendan O'Brien Mix
album                                          Pearl Jam (2017 Mix)
duration                                                     232186
listener_count                                                  989
mbid                                                            NaN
playcount                                                      1690
url               https://www.last.fm/music/pearl%2bjam/_/life%2...
info                                                             {}
date                                             01 Jan 2018, 11:06
timestamp                                                1514804801
track             PlayedTrack(track=pylast.Track('Pearl Jam', "L...
Name: 14811, dtype: object

#### __Indeed the first song is Life Wasted__

In [15]:
ranges = ['short_term', 'medium_term', 'long_term']
top_artists = set()
for a_range in ranges:
    results = sp.current_user_top_artists(time_range=a_range, limit=50)
    for item in results['items']:
        top_artists.add(item['name'])
    results = sp.current_user_top_artists(time_range=a_range, limit=50, offset=49)
    for item in results['items']:
        top_artists.add(item['name'])
top_artists

{'Acollective',
 'Adrián Berra',
 'Almendra',
 'Alvy, Nacho Y Rubin',
 'Ana Prada',
 'Anderson .Paak',
 'Animals As Leaders',
 'Arcade Fire',
 'Arctic Monkeys',
 'Asaf Avidan',
 'Atoms For Peace',
 'BROCKHAMPTON',
 'Babasónicos',
 'Banda do Mar',
 'Barney Kessel',
 'Beirut',
 'Bersuit Vergarabat',
 'Bill Frisell',
 'Bob Dylan',
 'Bon Iver',
 'Bonobo',
 'Bourbon Sweethearts',
 'Bruno Mars',
 'Bulgara',
 'Café Tacvba',
 'Carlos Gardel',
 'Catupecu Machu',
 'Chance the Rapper',
 'Charly García',
 'Chris Cornell',
 'Coldplay',
 'David Bowie',
 'Death Cab for Cutie',
 'Diego Boneta',
 'Divididos',
 'Dream Theater',
 'Duke Ellington',
 'Ed Sheeran',
 'Edmundo Rivero',
 'El Kuelgue',
 'Ella Fitzgerald',
 'Elliott Smith',
 'Esperanza Spalding',
 'Father John Misty',
 'Fleet Foxes',
 'Frank Ocean',
 'George Ezra',
 'Gilad Hekselman',
 'Glass Animals',
 'Gorillaz',
 'Green Day',
 'Gustavo Cerati',
 'Hinds',
 'Illya Kuryaki & The Valderramas',
 'Izan Llunas',
 'Jack Johnson',
 'Jack White',
 'Jak

In [16]:
reverse_df = df.iloc[::-1]
for i, row in reverse_df.iterrows():
    if row['artist'] not in top_artists:
        print("Song n# %d of the year" % (len(df) - i))
        print(row['artist'])
        print("Played on %s" % str(row['date']))
        break

Song n# 74 of the year
Whitney
Played on 01 Jan 2018, 16:04


In [17]:
df[df['artist'] == 'Faces']

,artist,title,album,duration,listener_count,mbid,playcount,url,info,date,timestamp,track


Last.fm doesn't register one scrobble to my name for Faces.

In [18]:
artist = pylast.Artist("Radiohead", network, username='alexing10')
print("As a baseline, Radiohead has %d scrobbles." % artist.get_userplaycount())

As a baseline, Radiohead has 1172 scrobbles.


In [19]:
artist = pylast.Artist("Faces", network, username='alexing10')
print("Faces has %d scrobbles." % artist.get_userplaycount())

Faces has 0 scrobbles.


In [20]:
artist = pylast.Artist("Whitney", network, username='alexing10')
print("Whitney has %d scrobbles and %d plays this year" % (artist.get_userplaycount(),len(df[df['artist'] == 'Whitney'])))

Whitney has 13 scrobbles and 12 plays this year


#### __Faces was never reproduced. First discovery is Whitney.__

### 3) "Listened to 58776 minutes." <font color='green'></font>  <font color='red'>FALSE</font>

In [21]:
df.duration.astype('int').sum() / 1000 / 60

60572.71953333333

We have some lead with the tracks regarding date

In [22]:
minutes_total_to_december = df[399:].duration.astype('int').sum() / 1000 / 60
minutes_total_to_december

59057.221783333334

In [26]:
dif = np.abs(58776 - minutes_total_to_december)
minutes, seconds = divmod(dif * 60 , 60)
print("There's a difference of %d minutes and %d seconds" % (minutes, seconds))
amount_of_songs = int(np.round(dif * 60 * 1000 / mean))
print("By mean duration of songs listened to this year, there's and error of %d songs" % amount_of_songs)

There's a difference of 281 minutes and 13 seconds
By mean duration of songs listened to this year, there's and error of 69 songs


### 4) "You spent 41 hours with your favorite artist Jorge Drexler, and the pleasure was all theirs." <font color='green'></font>  <font color='red'>FALSE</font>

In [27]:
top_listened = df[399:].groupby(by=['artist']).agg('sum')['duration'].sort_values(ascending=False)[:10]
top_listened = top_listened / 1000 / 60 / 60 # ms / s / m -> h
top_listened

artist
Jorge Drexler      42.335542
Serú Girán         33.730778
Radiohead          33.452604
Arctic Monkeys     26.142524
Gustavo Cerati     19.803698
Charly García      19.138179
Vampire Weekend    18.478611
Sufjan Stevens     17.274734
Soda Stereo        16.740329
Anderson .Paak     15.226572
Name: duration, dtype: float64

In [28]:
hours, minutes = divmod(top_listened[0] * 60, 60)
print("The real time is %d hours and %d minutes" % (int(hours), int(minutes)))

The real time is 42 hours and 20 minutes
